### Import Libraries

In [10]:
import re
import string
import emoji
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from kiwipiepy import Kiwi
from sklearn.utils import shuffle

In [2]:
nltk.download("stopwords")
nltk.download("punkt")
kiwi = Kiwi()
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nimisha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/nimisha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
stopwords_en = set(stopwords.words("english"))
stopwords_kr = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

## Load Dataset

In [4]:
df_en = pd.read_csv("../data/english_only_data.csv")
df_ko = pd.read_csv("../data/korean_only_data.csv")

In [5]:
df_en["language"] = "en"
df_ko["language"] = "ko"

df = pd.concat([df_en, df_ko], ignore_index=True)
df = shuffle(df, random_state=70)
df.head(10)

,Comment,Sentiment,language
133506,맛있고 분위기도 좋아요! 야외에서도 식사 가능해서 날씨 좋으면 더 좋아요,positive,ko
189273,어느 수산시장에서나 나오면 먹을 수 있는 민어.의 맛집 서비스 업종 역사상 최악 ...,negative,ko
2925,wow wow! what a funky little fleet!!! too sassy,Positive,en
41958,The invention original purpose is the most Bri...,Positive,en
104357,양도 푸짐하고 소스 모자라면 리필도 가능해서 좋아요. 한 명이 먹기에는 좀 많은 양...,positive,ko
32398,I feel sorry for this guy because you can tell...,Negative,en
158051,맛의 고장에 이런 식당이 있는 게 불가사의함..,negative,ko
81396,I've never heard someone talk so beautifully a...,Positive,en
156946,평점 0점은 왜 없는 거죠... 맛 집이라 해서 꾸불꾸불 산길을 비포장도로 따라 내...,negative,ko
108275,솔직히 별 하나도 아깝네요. 팥빙수라고 시켰는데 우유도 아니고 연유로 범벅을 해 가...,negative,ko


In [6]:
df["Comment"].isnull().sum()

0

In [7]:
df["Sentiment"] = df["Sentiment"].replace({"Negative": "negative", "Positive": "positive"})
df["Sentiment"].value_counts()

Sentiment
positive    100000
negative    100000
Name: count, dtype: int64

In [8]:
df["Sentiment"] = df["Sentiment"].replace({"negative": 0, "positive": 1})
df["Sentiment"].value_counts()

/var/folders/06/m9zqwbpx0xv34f89bzg3s27c0000gn/T/ipykernel_49313/4115779967.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Sentiment"] = df["Sentiment"].replace({"negative": 0, "positive": 1})


Sentiment
1    100000
0    100000
Name: count, dtype: int64

### Traditional Preprocessing

In [ ]:
def preprocess_for_traditional(text, language):
    text = emoji.demojize(text) 
    text = text.lower()

    text = re.sub(r"[^a-zA-Z0-9가-힣\s]", "", text)

    if language == "en":
        tokens = nltk.word_tokenize(text, language="english", preserve_line=True)
        tokens = [stemmer.stem(t) for t in tokens if t not in stopwords_en] 
    elif language == "ko":
        tokens = [word.form for word in kiwi.tokenize(text)]
        tokens = [stemmer.stem(t) for t in tokens if t not in stopwords_kr]

    return " ".join(tokens)

In [12]:
df_traditional = df.copy()
df_traditional["clean_comment"] = df_traditional.apply(
    lambda row: preprocess_for_traditional(row["Comment"], row["language"]), axis=1
)

In [13]:
df_traditional.head()

,Comment,Sentiment,language,clean_comment
133506,맛있고 분위기도 좋아요! 야외에서도 식사 가능해서 날씨 좋으면 더 좋아요,1,ko,맛있 분위기 좋 어요 ! 야외 에서 식사 가능 어서 날씨 좋 으면 더 좋 어요
189273,어느 수산시장에서나 나오면 먹을 수 있는 민어.의 맛집 서비스 업종 역사상 최악 ...,0,ko,어느 수산 시장 에서 나 나오 면 먹 수 있 민어 . 맛집 서비스 업종 역사 상 최...
2925,wow wow! what a funky little fleet!!! too sassy,1,en,wow wow funki littl fleet sassi
41958,The invention original purpose is the most Bri...,1,en,invent origin purpos british thing ever saw today
104357,양도 푸짐하고 소스 모자라면 리필도 가능해서 좋아요. 한 명이 먹기에는 좀 많은 양...,1,ko,양 푸짐 소스 모자라 면 리필 가능 어서 좋 어요 . 명 먹 기 좀 많 양 나 먹 ...


In [14]:
df_traditional = df_traditional[["clean_comment", "Sentiment", "language"]]
df_traditional.rename(
    columns={"clean_comment": "Comment", "language": "Language"}, inplace=True
)
df_traditional.head()

,Comment,Sentiment,Language
133506,맛있 분위기 좋 어요 ! 야외 에서 식사 가능 어서 날씨 좋 으면 더 좋 어요,1,ko
189273,어느 수산 시장 에서 나 나오 면 먹 수 있 민어 . 맛집 서비스 업종 역사 상 최...,0,ko
2925,wow wow funki littl fleet sassi,1,en
41958,invent origin purpos british thing ever saw today,1,en
104357,양 푸짐 소스 모자라 면 리필 가능 어서 좋 어요 . 명 먹 기 좀 많 양 나 먹 ...,1,ko


In [15]:
df_traditional.to_csv("../data/traditional_preprocessed_data.csv", index=False)

### Preprocessing for Word2Vec

In [ ]:
def preprocess_for_word_embedding(text, language):
    text = emoji.replace_emoji(text, replace="")

    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9가-힣\s]", "", text)

    if language == "en":
        tokens = nltk.word_tokenize(text, language="english", preserve_line=True)
        tokens = [t for t in tokens if t not in stopwords_en]
    elif language == "ko":
        tokens = [word.form for word in kiwi.tokenize(text)]
        tokens = [t for t in tokens if t not in stopwords_kr]

    return tokens

In [19]:
df_word_embedding = df.copy()
df_word_embedding["clean_comment"] = df_word_embedding.apply(
    lambda row: preprocess_for_word_embedding(row["Comment"], row["language"]), axis=1
)

In [16]:
df_word_embedding.head()

,Comment,Sentiment,language,clean_comment
133506,맛있고 분위기도 좋아요! 야외에서도 식사 가능해서 날씨 좋으면 더 좋아요,1,ko,"[맛있, 분위기, 좋, 어요, 야외, 에서, 식사, 가능, 어서, 날씨, 좋, 으면..."
189273,어느 수산시장에서나 나오면 먹을 수 있는 민어.의 맛집 서비스 업종 역사상 최악 ...,0,ko,"[어느, 수산, 시장, 에서, 나, 나오, 면, 먹, 수, 있, 민어, 맛집, 서비..."
2925,wow wow! what a funky little fleet!!! too sassy,1,en,"[wow, wow, funky, little, fleet, sassy]"
41958,The invention original purpose is the most Bri...,1,en,"[invention, original, purpose, british, thing,..."
104357,양도 푸짐하고 소스 모자라면 리필도 가능해서 좋아요. 한 명이 먹기에는 좀 많은 양...,1,ko,"[양, 푸짐, 소스, 모자라, 면, 리필, 가능, 어서, 좋, 어요, 명, 먹, 기..."


In [17]:
df_word_embedding = df_word_embedding[["clean_comment", "Sentiment", "language"]]
df_word_embedding.rename(
    columns={"clean_comment": "Comment", "language": "Language"}, inplace=True
)
df_word_embedding.head()

,Comment,Sentiment,Language
133506,"[맛있, 분위기, 좋, 어요, 야외, 에서, 식사, 가능, 어서, 날씨, 좋, 으면...",1,ko
189273,"[어느, 수산, 시장, 에서, 나, 나오, 면, 먹, 수, 있, 민어, 맛집, 서비...",0,ko
2925,"[wow, wow, funky, little, fleet, sassy]",1,en
41958,"[invention, original, purpose, british, thing,...",1,en
104357,"[양, 푸짐, 소스, 모자라, 면, 리필, 가능, 어서, 좋, 어요, 명, 먹, 기...",1,ko


In [18]:
df_word_embedding.to_csv("../data/embedding_preprocessed_data.csv", index=False)